### Introduction

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

In [33]:
prompt = "I want to open a restaurant for Nepali food. Suggest a fancy name for the restaurant. Give me a name only."
response = llm.invoke(prompt)
print(response)

content='Yak Cha Bar' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 61, 'total_tokens': 66, 'completion_time': 0.023298345, 'prompt_time': 0.003681218, 'queue_time': 0.052217442, 'total_time': 0.026979563}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--0f94ae31-82b3-4c38-809f-fff30b2f5d99-0' usage_metadata={'input_tokens': 61, 'output_tokens': 5, 'total_tokens': 66}


In [12]:
from langchain.prompts import PromptTemplate

prompt_template_resturant = PromptTemplate(
    input_variables = ['nationality'],
    template = "I want to open a restaurant for {nationality} food. Suggest a fancy name for the restaurant. Give me just a name."
)


prompt_template_resturant.format(nationality="Nepali")

'I want to open a restaurant for Nepali food. Suggest a fancy name for the restaurant. Give me just a name.'

In [31]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_resturant)
chain.run("Nepali")

'"Kathmandu Oasis"'

### Sequence Chain

In [27]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

prompt_template_name = PromptTemplate(
    input_variables = ['nationality'],
    template = "I want to open a restaurant for {nationality} food. Suggest a fancy name for the restaurant. Give me just a name."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)


prompt_template_items = PromptTemplate(
    input_variables = ['resturant name'],
    template = "Make up some menu items for {resturant name}. Return it as a comma seperated list."
)

items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [35]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, items_chain])

response = chain.run("Nepali")

print(response)

Here are some menu items for "Kathmandu Palace":

Chicken Tikka Masala, Vegetable Samosas, Lamb Korma, Saag Aloo, Palak Paneer, Momo Platter, Nepal Chicken Curry, Gorkhali Lamb Chops, Garlic Naan, Vegetable Biryani, Tandoori Chicken, Kachoris, Kathmandu Veggie Wrap.

If you want me to categorize them into different sections (appetizers, main courses, sides, etc.), I can do that as well. Let me know!


**To show the intermediate output**

In [41]:
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant"
)

prompt_template_name = PromptTemplate(
    input_variables = ['nationality'],
    template = "I want to open a restaurant for {nationality} food. Suggest a fancy name for the restaurant. Give me just a name."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="resturant_name")


prompt_template_items = PromptTemplate(
    input_variables = ['resturant_name'],
    template = "Make up some menu items for {resturant_name}. Return it as a comma seperated list."
)

items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [44]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains = [name_chain, items_chain],
                              input_variables = ['nationality'], 
                              output_variables = ['resturant_name', "menu_items"])

response = chain.invoke({"nationality": "Nepali"})

print(response)

{'nationality': 'Nepali', 'resturant_name': '"Rashtradhwaja"', 'menu_items': 'Here\'s a list of menu items for "Rashtradhwaja", a fictional Indian restaurant:\n\nRajasthani Laal Maans, Chicken Tikka Tandoori, Palak Paneer, Amritsari Kulche, Baingan Bharta, Hyderabadi Haleem, Malabar Fish Curry, Mattar Paneer, Dahi Vada, Aloo Gobhi, Mysore Masala Dosa, Chicken Biryani.\n\nNote: \'Rashtradhwaja\' is the Hindi word for \'National Flag\'. The menu items are inspired by the diverse and rich culinary heritage of India.'}
